In [4]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel,RunnableConfig
from langchain_core.messages import AIMessage
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.tools.render import render_text_description
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from operator import itemgetter
from langchain.embeddings import SentenceTransformerEmbeddings
import json
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS,Chroma
from operator import itemgetter
import time
import grandalf

In [5]:
load_dotenv(find_dotenv("D:\LLM Courses\Master Langchain Udemy\.env"))

True

In [70]:
# llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llm=ChatOpenAI(model="gpt-3.5-turbo")

In [64]:
prompt=ChatPromptTemplate.from_messages(
    messages=[
        SystemMessagePromptTemplate.from_template(
            template="""
                Write out the following equation using algebraic symbols then solve it. Use the format
                EQUATION:...
                SOLUTION:...
            """
        ),
        HumanMessagePromptTemplate.from_template(
            template="{equation_statement}"
        )
    ]
)

In [65]:
statement="5 times x plus 3 times y equals 7; Negative 3 times x plus 5 times y equals 23"

In [26]:

chain={"equation_statement":RunnablePassthrough()}|prompt|llm|StrOutputParser()

In [27]:
print(chain.invoke(input=statement))

EQUATION:
5x + 3y = 7
-3x + 5y = 23

SOLUTION:
To solve this system of equations, we can use the elimination method. 
First, let's multiply the first equation by 3 and the second equation by 5 to make the coefficients of y the same:

15x + 9y = 21
-15x + 25y = 115

Now, add the two equations together to eliminate x:

15x + 9y + (-15x + 25y) = 21 + 115
34y = 136
y = 4

Now, substitute y = 4 back into the first equation to solve for x:

5x + 3(4) = 7
5x + 12 = 7
5x = -5
x = -1

Therefore, the solution to the system of equations is x = -1 and y = 4.


<h3> Using the Stop Keywords</h3>

In [28]:
chain={"equation_statement":RunnablePassthrough()}|prompt|llm.bind(stop="SOLUTION")|StrOutputParser()

In [29]:
print(chain.invoke(input=statement))

EQUATION: 
5x + 3y = 7  
-3x + 5y = 23  




In [73]:
tools=[
        {
            "type":"function",
            "function": 
                {
                    "name": "solver",
                    "description":"Formulates and Solves an Equation",
                    "parameters":
                        {
                            "type":"object",
                            "properties":
                                {
                                    "equation":
                                        {
                                            "type":"string",
                                            "description":"The Algebric Expression of the Equation"
                                        },
                                    "solution":
                                        {
                                            "type":"string",
                                            "description":"The Solution to the Equation"
                                        }
                                },
                            "required":["equation","solution"]
                        },
                },
        }
    ]

In [74]:
chain={"equation_statement":RunnablePassthrough()}|prompt|llm.bind(tools=tools)

In [75]:
chain.invoke(input=statement).pretty_print()

================================== Ai Message ==================================
Tool Calls:
  solver (call_HK15bm4VaPt1r9D12EGR1KW4)
 Call ID: call_HK15bm4VaPt1r9D12EGR1KW4
  Args:
    equation: 5x + 3y = 7
    solution: x = 2, y = 1
  solver (call_IEjtIIeaUkucZHdAyjleOVDv)
 Call ID: call_IEjtIIeaUkucZHdAyjleOVDv
  Args:
    equation: -3x + 5y = 23
    solution: x = -3, y = 4
